In [7]:
flex_title = "Indonesian Atmosphere"
flex_subtitle = "Nusantara Earth Observation Network"

In [8]:
import ee
import os

# Interactive Python
import ipyleaflet
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date, timedelta, datetime

# Initialize Earth Engine token
ee_token = os.environ.get("EARTHENGINE_TOKEN")
if ee_token is not None:
    credential_file_path = os.path.expanduser("~/.config/earthengine/")
    if not os.path.exists(credential_file_path):
        credential = '{"refresh_token":"%s"}' % ee_token
        os.makedirs(credential_file_path, exist_ok=True)
        with open(credential_file_path + "credentials", "w") as file:
            file.write(credential)

ee.Initialize()

## Section

### Parameters

In [9]:
obs_label = widgets.Label(value='Date:')
obs_var = widgets.DatePicker(
    value=date.today() - timedelta(days=2)
)

param_label = widgets.Label(value='Pollutant:')
param_var = widgets.Dropdown(
    options=[
        'Nitrogen Dioxide', 'Carbon Monoxide', 'Sulphur Dioxide'
    ],
    value='Nitrogen Dioxide'
)

inputs = widgets.VBox([
    obs_label, obs_var, param_label, param_var
])
display(inputs)

### About

Nitrogen Dioxide column number density obtained by the TROPOMI to visualize the difference of emission pre-PPKM (left) and chosen date (right).

For more information, contact:
[Josef Matondang](mailto:admin@josefmtd.com)

## Column

### Sentinel-5P TROPOMI Measurements

In [10]:
trace_gases = {
    'Nitrogen Dioxide' : {
        'short' : 'NO2',
        'band' : 'tropospheric_NO2_column_number_density',
        'min' : 0.0,
        'max' : 200.0,
        'multiplier' : 1e6
    },
    'Carbon Monoxide' : {
        'short' : 'CO',
        'band' : 'CO_column_number_density',
        'min' : 0.0,
        'max' : 0.05,
        'multiplier' : 1e0
    },
    'Sulphur Dioxide' : {
        'short' : 'SO2',
        'band' : 'SO2_column_number_density',
        'min' : 0.0,
        'max' : 500.0,
        'multiplier' : 1e6
    },
    'Ozone' : {
        'short' : 'O3',
        'band' : 'O3_column_number_density',
        'min' : 0.12,
        'max' : 0.15,
        'multiplier' : 1e0
    }
}

def create_ee_tile_layer(image, vis_params, name):
    map_id_dict = image.getMapId(vis_params)
    tile_layer = ipyleaflet.TileLayer(
        url=map_id_dict["tile_fetcher"].url_format,
        attribution="European Space Agency, Google Earth Engine",
        name=name,
        opacity=1.0,
        shown=True
    )
    return tile_layer

def on_value_change(change):
    obs = obs_var.value
    param = param_var.value
    
    with out:
        gas_name = trace_gases[param]['short']
        band_name = trace_gases[param]['band']
        end = obs + timedelta(days=1)
        baseline = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_{}'.format(gas_name)) \
            .filterDate('2021-05-24', '2021-06-06') \
            .select(band_name)

        nrti = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_{}'.format(gas_name)) \
            .filterDate(obs.isoformat(), end.isoformat()) \
            .select(band_name)

        Map = ipyleaflet.Map(center=(0,120), zoom=5,
            min_zoom=5, max_zoom=12, 
            scroll_wheel_zoom=True,
            basemap=ipyleaflet.basemaps.CartoDB.Positron
        )

        vmin = trace_gases[param]["min"]
        vmax = trace_gases[param]["max"]
        multiplier = trace_gases[param]["multiplier"]
        palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']

        vis_params = {
            'min': vmin,
            'max': vmax,
            'palette': palette
        }

        baseline_vis = baseline.mean().multiply(multiplier) 
        nrti_vis = nrti.mean().multiply(multiplier)

        left = create_ee_tile_layer(baseline_vis, vis_params, 'Pre-PPKM')
        right = create_ee_tile_layer(nrti_vis, vis_params, 'Observed')

        control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
        Map.add_control(control)

        if change is not None:
            clear_output(wait=True)

        display(Map)

In [11]:
out = widgets.Output()
obs_var.observe(on_value_change, names="value")
param_var.observe(on_value_change, names="value")

on_value_change(None)
display(out)

Output()